In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [9]:
original = pd.read_csv('airlinedelaycauses_DelayedFlights.csv')

In [10]:
original.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
data = original.copy()

In [12]:
data = data.rename(columns={'Unnamed: 0': 'ID'})

In [19]:
data['DayofMonth'].max()

31

In [22]:
# merge Year, Month, DayofMonth and  DayOfWeek into a single Date column
data['Date'] = pd.to_datetime(dict(
    year=data['Year'],
    month=data['Month'],
    day=data['DayofMonth']
))


In [24]:
data.drop(columns=['Year', 'Month', 'DayofMonth'], inplace=True)

In [28]:
data.rename(columns={'CRSDepTime': 'ScheduledDepTime', 'DepTime' : 'ActualDepTime', 'CRSArrTime': 'ScheduledArrTime', 'ArrTime': 'ActualArrTime', 'CRSElapsedTime' : 'ScheduledElapsedTime'}, inplace=True)

In [45]:
data['CancellationReason'] = data['CancellationCode'].map({
    'N': 'Not Cancelled',
    'A': 'Carrier',
    'B': 'Weather',
    'C': 'NAS'
})
data.drop(columns=['CancellationCode'], inplace=True)

In [50]:
data.columns

Index(['ID', 'DayOfWeek', 'ActualDepTime', 'ScheduledDepTime', 'ActualArrTime',
       'ScheduledArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'ScheduledElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay', 'Date', 'CancellationReason'],
      dtype='object')

In [55]:
data['Diverted']

0          0
1          0
2          0
3          0
4          0
          ..
1936753    0
1936754    0
1936755    0
1936756    0
1936757    0
Name: Diverted, Length: 1936758, dtype: int64

In [61]:
data['ScheduledDepTime']

0          1955
1           735
2           620
3          1755
4          1915
           ... 
1936753    1220
1936754     600
1936755     847
1936756    1240
1936757    1103
Name: ScheduledDepTime, Length: 1936758, dtype: int64

In [ ]:
for col in ['ActualDepTime', 'ScheduledDepTime', 'ActualArrTime', 'ScheduledArrTime']:
    times = (
        data[col]
        .fillna(0)
        .astype(int)
        .astype(str)
        .str.zfill(4)
        .replace('2400', '0000')
    )
    data[col] = pd.to_datetime(times, format='%H%M', errors='coerce').dt.time


In [69]:
data['Date'] = pd.to_datetime(data['Date'])

In [72]:
categorical_cols = ['UniqueCarrier', 'Origin', 'Dest', 'CancellationReason', 'DayOfWeek']
data[categorical_cols] = data[categorical_cols].astype('category')

In [76]:
data['Cancelled'] = data['Cancelled'].astype(bool)
data['Diverted'] = data['Diverted'].astype(bool)

In [ ]:
delay_cols = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
data[delay_cols] = data[delay_cols].fillna(0)

In [109]:
data.isnull().sum()

ID                         0
DayOfWeek                  0
ActualDepTime              0
ScheduledDepTime           0
ActualArrTime              0
ScheduledArrTime           0
UniqueCarrier              0
FlightNum                  0
TailNum                    5
ActualElapsedTime       8387
ScheduledElapsedTime     198
AirTime                 8387
ArrDelay                8387
DepDelay                   0
Origin                     0
Dest                       0
Distance                   0
TaxiIn                  7110
TaxiOut                  455
Cancelled                  0
Diverted                   0
CarrierDelay               0
WeatherDelay               0
NASDelay                   0
SecurityDelay              0
LateAircraftDelay          0
Date                       0
CancellationReason         0
dtype: int64

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 28 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ID                    int64         
 1   DayOfWeek             category      
 2   ActualDepTime         object        
 3   ScheduledDepTime      object        
 4   ActualArrTime         object        
 5   ScheduledArrTime      object        
 6   UniqueCarrier         category      
 7   FlightNum             int64         
 8   TailNum               object        
 9   ActualElapsedTime     float64       
 10  ScheduledElapsedTime  float64       
 11  AirTime               float64       
 12  ArrDelay              float64       
 13  DepDelay              float64       
 14  Origin                category      
 15  Dest                  category      
 16  Distance              int64         
 17  TaxiIn                float64       
 18  TaxiOut               float64       
 19  

In [112]:
data.to_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv', index=False)

In [3]:
data_cleaned = pd.read_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv')

# Visualizations

In [8]:
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Load and preprocess data
data = pd.read_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv')

# Preprocess columns
# Handle ScheduledDepTime - it might be in HH:MM:SS format or HHMM format
if data['ScheduledDepTime'].dtype == 'object':
    # If it's a time string like "19:55:00", extract hour
    try:
        data['Hour'] = pd.to_datetime(data['ScheduledDepTime'], format='%H:%M:%S', errors='coerce').dt.hour
    except:
        # If that fails, try parsing as string and extract hour part
        data['Hour'] = data['ScheduledDepTime'].astype(str).str.split(':').str[0]
        data['Hour'] = pd.to_numeric(data['Hour'], errors='coerce').fillna(0).astype(int)
else:
    # If it's numeric (like 1955 for 19:55)
    data['ScheduledDepTime'] = pd.to_numeric(data['ScheduledDepTime'], errors='coerce')
    data['Hour'] = (data['ScheduledDepTime'] // 100).fillna(0).astype(int)

# Ensure hour is in valid range (0-23)
data['Hour'] = data['Hour'].clip(0, 23)
data['DayOfWeek'] = data['DayOfWeek'].astype(int)
data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.month
data['MonthName'] = data['Date'].dt.strftime('%B')

# Day of week mapping
day_names = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 
             5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
data['DayName'] = data['DayOfWeek'].map(day_names)

# Initialize Dash app
app = Dash(__name__)

# Color scheme
colors = {
    'background': '#f8f9fa',
    'text': '#2c3e50',
    'primary': '#3498db',
    'secondary': '#e74c3c'
}

# Layout
app.layout = html.Div(style={'backgroundColor': colors['background'], 'padding': '20px'}, children=[
    html.H1("✈️ Flight Delay Analytics Dashboard", 
            style={'textAlign': 'center', 'color': colors['text'], 'marginBottom': '30px'}),
    
    # Control Panel
    html.Div(style={'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '10px', 
                    'marginBottom': '20px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'}, children=[
        html.H3("Filters & Controls", style={'color': colors['text'], 'marginBottom': '15px'}),
        
        html.Div([
            html.Div([
                html.Label("Delay Type:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='delay-type-dropdown',
                    options=[
                        {'label': '🛬 Arrival Delay', 'value': 'ArrDelay'},
                        {'label': '🛫 Departure Delay', 'value': 'DepDelay'}
                    ],
                    value='ArrDelay',
                    clearable=False
                )
            ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%'}),
            
            html.Div([
                html.Label("Select Airlines:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='airline-filter',
                    options=[{'label': carrier, 'value': carrier} 
                            for carrier in sorted(data['UniqueCarrier'].unique())],
                    value=sorted(data['UniqueCarrier'].unique()),
                    multi=True
                )
            ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%'}),
            
            html.Div([
                html.Label("Date Range:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.DatePickerRange(
                    id='date-range',
                    start_date=data['Date'].min(),
                    end_date=data['Date'].max(),
                    display_format='YYYY-MM-DD'
                )
            ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%'}),
            
            html.Div([
                html.Label("Min Delay (minutes):", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Slider(
                    id='delay-threshold',
                    min=0,
                    max=120,
                    step=15,
                    value=15,
                    marks={i: f'{i}' for i in range(0, 121, 30)},
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ], style={'width': '23%', 'display': 'inline-block'})
        ])
    ]),
    
    # Summary Statistics
    html.Div(id='summary-stats', style={'marginBottom': '20px'}),
    
    # Main Charts
    html.Div([
        html.Div([
            dcc.Graph(id='delay-over-time')
        ], style={'width': '49%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            dcc.Graph(id='delay-distribution')
        ], style={'width': '49%', 'display': 'inline-block'})
    ]),
    
    html.Div([
        html.Div([
            dcc.Graph(id='airline-delay-bar')
        ], style={'width': '49%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            dcc.Graph(id='route-analysis')
        ], style={'width': '49%', 'display': 'inline-block'})
    ]),
    
    html.Div([
        html.Div([
            dcc.Graph(id='heatmap-delay')
        ], style={'width': '49%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            dcc.Graph(id='delay-causes')
        ], style={'width': '49%', 'display': 'inline-block'})
    ]),
    
    html.Div([
        html.Div([
            dcc.Graph(id='cancellation-pie')
        ], style={'width': '49%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            dcc.Graph(id='hourly-pattern')
        ], style={'width': '49%', 'display': 'inline-block'})
    ])
])

# Callbacks
@callback(
    [Output('summary-stats', 'children'),
     Output('delay-over-time', 'figure'),
     Output('delay-distribution', 'figure'),
     Output('airline-delay-bar', 'figure'),
     Output('route-analysis', 'figure'),
     Output('heatmap-delay', 'figure'),
     Output('delay-causes', 'figure'),
     Output('cancellation-pie', 'figure'),
     Output('hourly-pattern', 'figure')],
    [Input('delay-type-dropdown', 'value'),
     Input('airline-filter', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('delay-threshold', 'value')]
)
def update_dashboard(selected_delay, selected_airlines, start_date, end_date, delay_threshold):
    # Filter data
    filtered_data = data[
        (data['UniqueCarrier'].isin(selected_airlines)) &
        (data['Date'] >= start_date) &
        (data['Date'] <= end_date)
    ].copy()
    
    # Check if filtered data is empty
    if len(filtered_data) == 0:
        empty_fig = go.Figure()
        empty_fig.add_annotation(
            text="No data available for selected filters. Please adjust your selections.",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False,
            font=dict(size=16, color='red')
        )
        empty_fig.update_layout(template='plotly_white')
        
        summary = html.Div(style={'textAlign': 'center', 'padding': '20px', 'backgroundColor': '#ffebee', 
                                  'borderRadius': '10px', 'color': '#c62828'}, 
                          children=[html.H3("⚠️ No flights match the selected filters")])
        
        return summary, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig
    
    # Summary Statistics
    total_flights = len(filtered_data)
    delayed_flights = len(filtered_data[filtered_data[selected_delay] > delay_threshold])
    avg_delay = filtered_data[selected_delay].mean()
    max_delay = filtered_data[selected_delay].max()
    cancelled = filtered_data['Cancelled'].sum()
    
    delay_pct = (delayed_flights/total_flights*100) if total_flights > 0 else 0
    
    summary = html.Div(style={'display': 'flex', 'justifyContent': 'space-around'}, children=[
        create_stat_card("Total Flights", f"{total_flights:,}", "✈️"),
        create_stat_card("Delayed Flights", f"{delayed_flights:,} ({delay_pct:.1f}%)", "⏰"),
        create_stat_card("Avg Delay", f"{avg_delay:.1f} min", "📊"),
        create_stat_card("Max Delay", f"{max_delay:.0f} min", "🔴"),
        create_stat_card("Cancelled", f"{cancelled:,}", "❌")
    ])
    
    # 1. Delay over time (with trend)
    delay_time = filtered_data.groupby('Date')[selected_delay].agg(['mean', 'count']).reset_index()
    fig_line = go.Figure()
    fig_line.add_trace(go.Scatter(x=delay_time['Date'], y=delay_time['mean'],
                                  mode='lines', name='Daily Average',
                                  line=dict(color=colors['primary'], width=2)))
    fig_line.add_trace(go.Scatter(x=delay_time['Date'], y=delay_time['mean'].rolling(7).mean(),
                                  mode='lines', name='7-Day Moving Avg',
                                  line=dict(color=colors['secondary'], width=3, dash='dash')))
    fig_line.update_layout(title=f'Average {selected_delay} Over Time',
                          xaxis_title='Date', yaxis_title='Minutes',
                          hovermode='x unified', template='plotly_white')
    
    # 2. Delay Distribution
    fig_dist = px.histogram(filtered_data[filtered_data[selected_delay] > 0], 
                           x=selected_delay, nbins=50,
                           title=f'{selected_delay} Distribution',
                           labels={selected_delay: 'Delay (minutes)'},
                           color_discrete_sequence=[colors['primary']])
    fig_dist.update_layout(template='plotly_white', showlegend=False)
    
    # 3. Airline delay comparison
    airline_delay = filtered_data.groupby('UniqueCarrier').agg({
        selected_delay: 'mean',
        'ID': 'count'
    }).reset_index()
    airline_delay.columns = ['UniqueCarrier', 'AvgDelay', 'FlightCount']
    airline_delay = airline_delay.sort_values('AvgDelay', ascending=True)
    
    fig_bar = go.Figure(data=[
        go.Bar(x=airline_delay['AvgDelay'], y=airline_delay['UniqueCarrier'],
               orientation='h',
               text=airline_delay['AvgDelay'].round(1),
               textposition='auto',
               marker_color=colors['primary'],
               customdata=airline_delay['FlightCount'],
               hovertemplate='<b>%{y}</b><br>Avg Delay: %{x:.1f} min<br>Flights: %{customdata}<extra></extra>')
    ])
    fig_bar.update_layout(title=f'Average {selected_delay} per Airline',
                         xaxis_title='Minutes', yaxis_title='Airline',
                         template='plotly_white')
    
    # 4. Top 10 Routes by delay
    filtered_data['Route'] = filtered_data['Origin'] + ' → ' + filtered_data['Dest']
    route_delay = filtered_data.groupby('Route').agg({
        selected_delay: 'mean',
        'ID': 'count'
    }).reset_index()
    route_delay.columns = ['Route', 'AvgDelay', 'FlightCount']
    route_delay = route_delay[route_delay['FlightCount'] >= 100].nlargest(10, 'AvgDelay')
    
    fig_route = px.bar(route_delay, x='AvgDelay', y='Route',
                      title='Top 10 Most Delayed Routes (min 100 flights)',
                      orientation='h',
                      color='AvgDelay',
                      color_continuous_scale='Reds',
                      labels={'AvgDelay': 'Avg Delay (min)'})
    fig_route.update_layout(template='plotly_white', showlegend=False)
    
    # 5. Heatmap: day of week vs hour
    heatmap_data = filtered_data.groupby(['DayOfWeek', 'Hour'])[selected_delay].mean().reset_index()
    heatmap_pivot = heatmap_data.pivot(index='DayOfWeek', columns='Hour', values=selected_delay)
    
    fig_heatmap = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=[day_names.get(i, i) for i in heatmap_pivot.index],
        colorscale='RdYlGn_r',
        hovertemplate='Hour: %{x}<br>Day: %{y}<br>Avg Delay: %{z:.1f} min<extra></extra>'
    ))
    fig_heatmap.update_layout(title=f'{selected_delay} Heatmap: Day of Week vs Hour',
                             xaxis_title='Hour of Day', yaxis_title='Day of Week',
                             template='plotly_white')
    
    # 6. Delay causes breakdown
    delay_causes = filtered_data[filtered_data[selected_delay] > delay_threshold][
        ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
    ].sum()
    
    fig_causes = go.Figure(data=[
        go.Bar(x=delay_causes.index, y=delay_causes.values,
               marker_color=['#3498db', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c'])
    ])
    fig_causes.update_layout(title='Total Delay Minutes by Cause',
                            xaxis_title='Cause', yaxis_title='Total Minutes',
                            template='plotly_white', showlegend=False)
    
    # 7. Cancellation reasons
    cancel_data = filtered_data[filtered_data['Cancelled'] == True]['CancellationReason'].value_counts()
    if len(cancel_data) > 0:
        fig_pie = px.pie(values=cancel_data.values, names=cancel_data.index,
                        title='Cancellation Reasons',
                        color_discrete_sequence=px.colors.qualitative.Set3)
    else:
        fig_pie = go.Figure()
        fig_pie.add_annotation(text="No Cancellations in Selected Range",
                              xref="paper", yref="paper",
                              x=0.5, y=0.5, showarrow=False)
        fig_pie.update_layout(title='Cancellation Reasons')
    fig_pie.update_layout(template='plotly_white')
    
    # 8. Hourly pattern
    hourly_pattern = filtered_data.groupby('Hour')[selected_delay].mean().reset_index()
    hourly_pattern = hourly_pattern.sort_values('Hour')
    
    fig_hourly = go.Figure(data=[
        go.Scatter(x=hourly_pattern['Hour'], y=hourly_pattern[selected_delay],
                  mode='lines+markers', fill='tozeroy',
                  line=dict(color=colors['primary'], width=3),
                  marker=dict(size=8),
                  hovertemplate='Hour: %{x}:00<br>Avg Delay: %{y:.1f} min<extra></extra>')
    ])
    fig_hourly.update_layout(title='Average Delay by Hour of Day',
                            xaxis_title='Hour of Day (0-23)', yaxis_title='Avg Delay (min)',
                            xaxis=dict(tickmode='linear', tick0=0, dtick=2, range=[-0.5, 23.5]),
                            template='plotly_white', showlegend=False)
    
    return summary, fig_line, fig_dist, fig_bar, fig_route, fig_heatmap, fig_causes, fig_pie, fig_hourly

def create_stat_card(label, value, icon):
    return html.Div(style={
        'backgroundColor': 'white',
        'padding': '20px',
        'borderRadius': '10px',
        'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
        'textAlign': 'center',
        'width': '18%'
    }, children=[
        html.Div(icon, style={'fontSize': '32px', 'marginBottom': '10px'}),
        html.H4(value, style={'color': colors['primary'], 'margin': '10px 0'}),
        html.P(label, style={'color': colors['text'], 'fontSize': '14px', 'margin': '0'})
    ])

if __name__ == '__main__':
    app.run(debug=True)